In [ ]:
# Import and init h2o lib. Running on java
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
h2o.init()

col_train1 = {
}
col_train2 = {
}
col_train3 = {
}
col_train4 = {
}
col_train5 = {
}

In [ ]:
# Reading csv to h2o dataframe 

train_data = pd.read_csv('old_data/feature_data/training_set_dropped_old.csv')
test_data = pd.read_csv('old_data/feature_data/testing_set_dropped_old.csv')


#set 1
train1 = train_data.drop(['lv4', 'municipality_size_group', 'num_stores_within_10km_and_same_lvl2', 'mean_revenue_for_municipality_size_group', 'grunnkrets_id', 'mean_revenue_lv4', 'chain_name', 'closest_competitor_lv2', 'lv1', 'nbr_people_per_store_in_grunnkrets'], axis=1)
test1 = test_data.drop(['lv4', 'municipality_size_group', 'num_stores_within_10km_and_same_lvl2', 'mean_revenue_for_municipality_size_group', 'grunnkrets_id', 'mean_revenue_lv4', 'chain_name', 'closest_competitor_lv2', 'lv1', 'nbr_people_per_store_in_grunnkrets'], axis=1)
#set 2
train2 = train_data.drop(['num_stores_within_20km_and_same_lvl2', 'median_revenue_lv4', 'name', 'busstops_within_50m', 'sales_channel_name', 'num_stores_within_5km_and_same_lvl3', 'mean_revenue_for_municipality_size_group', 'num_stores_within_100m_and_same_lvl2', 'closest_competitor_lv2', 'mean_revenue_lv4'], axis=1)
test2 = test_data.drop(['num_stores_within_20km_and_same_lvl2', 'median_revenue_lv4', 'name', 'busstops_within_50m', 'sales_channel_name', 'num_stores_within_5km_and_same_lvl3', 'mean_revenue_for_municipality_size_group', 'num_stores_within_100m_and_same_lvl2', 'closest_competitor_lv2', 'mean_revenue_lv4'], axis=1)
#set 3
train3 = train_data.drop(['mean_revenue_for_municipality_size_group', 'num_stores_within_500m_and_same_lvl2', 'num_stores_within_1km_and_same_lvl2', 'num_stores_within_1km_and_same_lvl3', 'sales_channel_name', 'busstops_within_400m', 'num_stores_within_20km_and_same_lvl3', 'mean_revenue_for_municipality_rev_group_lv2', 'lon', 'busstops_within_800m'], axis=1)
test3 = test_data.drop(['mean_revenue_for_municipality_size_group', 'num_stores_within_500m_and_same_lvl2', 'num_stores_within_1km_and_same_lvl2', 'num_stores_within_1km_and_same_lvl3', 'sales_channel_name', 'busstops_within_400m', 'num_stores_within_20km_and_same_lvl3', 'mean_revenue_for_municipality_rev_group_lv2', 'lon', 'busstops_within_800m'], axis=1)
#set 4
train4 = train_data.drop(['grunnkrets_id', 'mall_name', 'num_stores_within_1km_and_same_lvl2', 'plaace_hierarchy_id', 'num_stores_within_20km_and_same_lvl3', 'mean_revenue_lv3', 'st_dev_of_revenue_for_municipality_size_group', 'counts_gr_lv2', 'median_revenue_lv4', 'num_stores_within_100m_and_same_lvl3'], axis=1)
test4 = test_data.drop(['grunnkrets_id', 'mall_name', 'num_stores_within_1km_and_same_lvl2', 'plaace_hierarchy_id', 'num_stores_within_20km_and_same_lvl3', 'mean_revenue_lv3', 'st_dev_of_revenue_for_municipality_size_group', 'counts_gr_lv2', 'median_revenue_lv4', 'num_stores_within_100m_and_same_lvl3'], axis=1)
#set 5
train5 = train_data.drop(['busstops_within_400m', 'num_stores_within_20km_and_same_lvl3', 'num_stores_within_20km_and_same_lvl2', 'num_stores_within_5km_and_same_lvl3', 'counts_municipality_lv2', 'name', 'busstops_within_100m', 'nbr_people_per_store_in_grunnkrets', 'median_revenue_lv2', 'num_stores_within_10km_and_same_lvl2'], axis=1)
test5 = test_data.drop(['busstops_within_400m', 'num_stores_within_20km_and_same_lvl3', 'num_stores_within_20km_and_same_lvl2', 'num_stores_within_5km_and_same_lvl3', 'counts_municipality_lv2', 'name', 'busstops_within_100m', 'nbr_people_per_store_in_grunnkrets', 'median_revenue_lv2', 'num_stores_within_10km_and_same_lvl2'], axis=1)


# Save store ids, then remove this column
#store_ids = test_data['store_id'].to_numpy()
#ids = pd.DataFrame(store_ids)
#ids.rename(columns={0 :'id'}, inplace=True)
#
#train_data = train_data.drop('store_id', axis=1)
#test_data = test_data.drop('store_id', axis=1)
#
#train1 = h2o.H2OFrame(train1)
#train2 = h2o.H2OFrame(train2)
#train3 = h2o.H2OFrame(train3)
#train4 = h2o.H2OFrame(train4)
#train5 = h2o.H2OFrame(train5)
del col_train1['revenue']
del col_train2['revenue']
del col_train3['revenue']
del col_train4['revenue']
del col_train5['revenue']
#test1 = h2o.H2OFrame(test1)
#test2 = h2o.H2OFrame(test2)
#test3 = h2o.H2OFrame(test3)
#test4 = h2o.H2OFrame(test4)
#test5 = h2o.H2OFrame(test5)



In [ ]:
# Removing labels from train set
y = "revenue"
x1 = train1.columns
x2 = train1.columns
x3 = train1.columns
x4 = train1.columns
x5 = train1.columns
x1.remove(y)
x2.remove(y)
x3.remove(y)
x4.remove(y)
x5.remove(y)

In [ ]:
# Running fit/training on train set with cross validation n=5
aml1 = H2OAutoML(max_models = 5, seed = 2, stopping_metric = "RMSE", exclude_algos=["XGBoost", "DeepLearning"])
aml1.train(x = x1, y = y, training_frame = train1)

aml2 = H2OAutoML(max_models = 5, seed = 2, stopping_metric = "RMSE", exclude_algos=["XGBoost", "StackedEnsemble"])
aml2.train(x = x2, y = y, training_frame = train2)

aml3 = H2OAutoML(max_models = 5, seed = 2, stopping_metric = "RMSE", exclude_algos=["GBM", "DeepLearning"])
aml3.train(x = x3, y = y, training_frame = train3)

aml4 = H2OAutoML(max_models = 5, seed = 2, stopping_metric = "RMSE", exclude_algos=["XGBoost", "DRF"])
aml4.train(x = x4, y = y, training_frame = train4)

aml5 = H2OAutoML(max_models = 5, seed = 2, stopping_metric = "RMSE", exclude_algos=["GLM", "DeepLearning"])
aml5.train(x = x5, y = y, training_frame = train5)

In [ ]:
lb1 = aml1.leaderboard
lb2 = aml2.leaderboard
lb3 = aml3.leaderboard
lb4 = aml4.leaderboard
lb5 = aml5.leaderboard

In [ ]:
# Showing best perforimg models
lb1.head()
lb2.head()
lb3.head()
lb4.head()
lb5.head()

In [ ]:
preds1 = aml1.predict(test_data)
preds2 = aml2.predict(test_data)
preds3 = aml3.predict(test_data)
preds4 = aml4.predict(test_data)
preds5 = aml5.predict(test_data)

In [ ]:
#print variable importance
#corr = aml1.varimp(use_pandas = True)
#corr['sum'] = corr.sum(axis=1)
#corr = corr.sort_values(by=['sum'])
#corr.head(30)

In [ ]:
#preds = h2o.as_list(preds)
#print(preds)

In [ ]:
# Convert result to desired format

import pandas as pd
import numpy as np

weights = [0.4, 0.55, 0.45, 0.6, 0.5]
sum_weights = sum(weights)
weights = pd.DataFrame(weights)


pandas_preds1 = preds1.as_data_frame()
pandas_preds1.rename(columns={'predict' :'1'}, inplace=True)
pandas_preds2 = preds2.as_data_frame()
pandas_preds2.rename(columns={'predict' :'2'}, inplace=True)
pandas_preds3 = preds3.as_data_frame()
pandas_preds3.rename(columns={'predict' :'3'}, inplace=True)
pandas_preds4 = preds4.as_data_frame()
pandas_preds4.rename(columns={'predict' :'4'}, inplace=True)
pandas_preds5 = preds5.as_data_frame()
pandas_preds5.rename(columns={'predict' :'5'}, inplace=True)

pandas_preds = pandas_preds1.join(pandas_preds2).join(pandas_preds3).join(pandas_preds4).join(pandas_preds5)
pandas_preds['predicted'] = pandas_preds.dot(weights)/sum_weights


#pandas_preds.rename(columns={'predict' :'predicted'}, inplace=True)
pandas_preds['predicted'] = np.expm1(pandas_preds['predicted'])

def save_submission(pred):
    predicted = pd.DataFrame(pred, columns = ['predicted'])
    output = pd.concat([ids,predicted],axis=1)
    output.to_csv('h2o.csv', index=False)
    
save_submission(pandas_preds)



In [ ]:
#aml.explain(frame = train_data)